In [39]:
import random
import pandas as pd
import numpy as np
import copy

In [40]:
# загрузка данных
df = pd.read_excel('data_with_score_directions_roles.xlsx')
projects_df = pd.read_excel('projects_raw_data.xlsx')

In [41]:
# add version with pre-selected students

def data_preparation(df_students, df_projects):
   students = []
   students_df = df_students.reset_index()

   for _, student_df in students_df.iterrows():
      student = dict()
      student['st'] = student_df['Единая учетная запись СПбГУ (например, ST000000)']
      student['year'] = student_df['Year']
      student['season'] =  student_df['Season']
      student['score'] = student_df['score']
      #student['directions'] = eval(student_df['directions'])
      #student['roles'] = eval(student_df['roles'])
      students.append(student)
    
   projects = []
   projects_df = df_projects.reset_index()
   for _, project_df in projects_df.iterrows():
      project = dict()
      project['name'] = project_df['Проект']
      project['year'] = project_df['Год']
      project['season'] = project_df['Сезон']
      project['remainder'] = 0
      d = []
      for w in project_df['Направление'].split(sep=','):
         d.append(w.strip())
      #project['directions'] = d
      project['team'] = []
      c=[]

      '''for w in project_df['Состав команды'].split(sep=','):
         if ':' in w:
            student_st = w.split(sep=':')[1].strip()
            for s in students:
               if s['st'] == student_st:
                  s['assigned'] = True
                  project['team'].append(s)
                  role = w.split(sep=':')[0]
                  project[role.strip().lower()] = []
                  project[role.strip().lower()].append([s['st'], s['score'], s['year'], s['season']])
                  break
            c.append(role.strip().lower())
         else:   
            project[w.strip().lower()] = []
            c.append(w.strip().lower())
      project['compound'] = c'''
      
      projects.append(project)

   return students, projects


In [42]:
# Распределение по остаточному признаку

def distribute_students_random(students, projects, size=None):
    assigned_students = []
    distributed_projects = copy.deepcopy(projects)
    distributed_students = copy.deepcopy(students)
    for student in distributed_students:
        if ('assigned' in student.keys()) and (student['assigned'] == True):
            continue
        else: 
            student['assigned'] = False
    
    if size is None: 
        size = len(students)//len(projects)

    remaining_students = [student for student in distributed_students if ('assigned' in student.keys()) and (student['assigned'] == False)]
    for project in distributed_projects:
        project['remainder'] = size
        while project['remainder'] > 0 and len(remaining_students) > 0:
            student = remaining_students.pop(random.choice(range(len(remaining_students))))
            project['team'].append([student['st'], student['score'], student['year'], student['season']])
            project['remainder'] -= 1
            student['assigned'] = True
            assigned_students.append(student)  # Добавить студента в список назначенных        

        if project['remainder'] == 0: 
            project['status'] = 'complete'
        
        project['team_mean_score'] = np.mean([s[1] for s in project['team']])
    
    while len(remaining_students) > 0:
            student = remaining_students.pop(random.choice(range(len(remaining_students))))
            p = random.choice(range(len(distributed_projects)))
            distributed_projects[p]['team'].append([student['st'], student['score'], student['year'], student['season']])
            if len(distributed_projects[p]['team']) == 0: 
               distributed_projects[p]['remainder'] = size - 1
            else:
                distributed_projects[p]['remainder'] -= 1
            student['assigned'] = True
            assigned_students.append(student)  
            distributed_projects[p]['team_mean_score'] = np.mean([s[1] for s in project['team']])

    
    return distributed_students, distributed_projects
            
    

In [43]:
#Пример для 2023 года Осень

df_students_2023 = df[(df['Year'] == 2023) & (df['Season'] == 'Осень')]
df_projects_2023 = projects_df[(projects_df['Год'] == 2023) & (projects_df['Сезон'] == 'Осень')]

In [44]:
students, projects = data_preparation(df_students_2023, df_projects_2023)
#team_size = 8
#s_23,p_23 = distribute_students_random(students, projects)

In [56]:
samples = []
for i in range(100):
    _,p_23 = distribute_students_random(students, projects)
    samples.append(p_23)


In [58]:
d = pd.DataFrame.from_records(samples)

In [65]:
d.to_excel('DistributionrandomSamples_23autumn.xlsx')